In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/nlp-getting-started/sample_submission.csv
/kaggle/input/nlp-getting-started/train.csv
/kaggle/input/nlp-getting-started/test.csv


In [2]:
#Load Dataset
df=pd.read_csv('/kaggle/input/nlp-getting-started/train.csv')
df.head()

,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1


In [3]:
import nltk
import re
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
nltk.download('stopwords')
nltk.download('punkt')
stop_words = set(stopwords.words('english'))
def tokenize_text(text):
    text=text.lower()
    text = re.sub(r'@\w+', '', text) #remove @mentions
    text = re.sub(r'[@\.\,\!\#\$\%\&\*\+\-\/\:\;<\=\>\?\@\[\\\]\^\_\`\{\|\}\~]+', '', text)
    words = nltk.word_tokenize(text)
    filtered_tokens = [word for word in words if not word in stop_words]
    return filtered_tokens
df['text'].fillna('')
df['tokenized_text'] = df['text'].apply(tokenize_text)

[nltk_data] Downloading package stopwords to /usr/share/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /usr/share/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [4]:
from sklearn import feature_extraction, linear_model, model_selection, preprocessing
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer

In [5]:
y=df['target']
X = df['tokenized_text'].apply(lambda x: ' '.join(x))


In [6]:
max_len = 500
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.preprocessing.text import one_hot

vocab_size = 500
encoded_docs = [one_hot(d, vocab_size) for d in X]
encoded_docs = pad_sequences(encoded_docs, maxlen=max_len)

In [7]:
# split the data into train and test
X_train, X_test, y_train, y_test = train_test_split(encoded_docs, y, test_size=0.2, random_state=1)


In [8]:
from keras.models import Sequential
from keras.layers import Embedding, Conv1D, GlobalMaxPooling1D, Dense
from keras.layers import Dropout
max_words = 500
model = Sequential()
model.add(Embedding(max_words, 512, input_length=max_len))
model.add(Dropout(0.42))
model.add(Conv1D(32, 4, activation='relu'))
# model.add(Conv1D(32, 4, activation='relu'))
# model.add(Dropout(0.01))
# model.add(Conv1D(32, 4, activation='relu'))
model.add(GlobalMaxPooling1D())
model.add(Dense(1, activation='sigmoid'))

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [9]:
model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=1, batch_size=256)
model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=1, batch_size=128)
model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=1, batch_size=64)
model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=3, batch_size=16)
# model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=3, batch_size=256)

96/96 [==============================] - 48s 498ms/step - loss: 0.5217 - accuracy: 0.7442 - val_loss: 0.5829 - val_accuracy: 0.6907
Epoch 1/3
381/381 [==============================] - 45s 117ms/step - loss: 0.4924 - accuracy: 0.7672 - val_loss: 0.5860 - val_accuracy: 0.7019
Epoch 2/3
381/381 [==============================] - 44s 116ms/step - loss: 0.4131 - accuracy: 0.8190 - val_loss: 0.6003 - val_accuracy: 0.7262
Epoch 3/3
381/381 [==============================] - 44s 116ms/step - loss: 0.3403 - accuracy: 0.8586 - val_loss: 0.6366 - val_accuracy: 0.7039


In [10]:
df_test=pd.read_csv('/kaggle/input/nlp-getting-started/test.csv')
df_test['text'].fillna('')
df_test['tokenized_text'] = df_test['text'].apply(tokenize_text)

In [11]:
X = df_test['tokenized_text'].apply(lambda x: ' '.join(x))

In [12]:
encoded_docs = [one_hot(d, vocab_size) for d in X]
encoded_docs = pad_sequences(encoded_docs, maxlen=max_len)

In [13]:
model_predict=model.predict(encoded_docs)

102/102 [==============================] - 3s 30ms/step


In [14]:
model_predict

array([[0.4595131 ],
       [0.65242213],
       [0.98913074],
       ...,
       [0.93351024],
       [0.5784366 ],
       [0.61458385]], dtype=float32)

In [15]:
# result=pd.DataFrame({'id': df_test['id'], 'target': [model_predict]})

res = pd.DataFrame(model_predict, columns=["target"])
res['id']=df_test['id']


def convert_to_binary(x):
    if x >= 0.5:
        return 1
    else:
        return 0

res['target']=res['target'].apply(convert_to_binary)

In [16]:
res.head()

,target,id
0,0,0
1,1,2
2,1,3
3,1,9
4,1,11


In [17]:
res.to_csv('/kaggle/working/submissions_CNN.csv',index=False)